In [ ]:
from Functions import*
import matplotlib.pyplot as plt

# connect to the LCR meter
ser = DeviceConnect('COM10')

try:
    params = input('Input gel,  plasticizer content, and test i.e. P8 PVC Impedance')
    area = input('Input the gel area in mm i.e. 10.2')
    thickness = input('Input the gel thickness in mm i.e. 1')

    # ser - serial connection
    # freq - frequency limits [lower, upper] (Hz)
    # measurement - choose a measurement below. input the number
    #   CSQ/0  | CSD/1  | CSR/2  | CPQ/3  | CPD/4  | CPR/5 |
    #   CPG/6  | LSQ/7  | LSD/8  | LSR/9  | LPQ/10 | LPD/11|
    #   LPR/12 | LPG/13 | ZTH/14 | YTH/15 | RX/16  | GB/17 |
    # voltage - the voltage applied between electrodes either 0.5 or 1 (V)

    Data = Experiment(ser, frequencies = [20, 10000], measurement = 14, voltage = 1)
    DeviceDisconnect(ser)
    DataExport(params, Data, f'{area}, {thickness}, {1} - Area, thickness, voltage')

except: 
    print('Test Aborted')
    DeviceDisconnect(ser)     


Test completed ✅




In [ ]:
import os; import numpy as np

files = [i for i in os.listdir('Data') if i.endswith('.csv')]

for i in files:
    data = np.loadtxt(i, delimiter = ',')


In [ ]:
import os


def ComplexResistivity(thickness, diameter, dataFrame):
    area = np.pi*diameter**2/4 
    angularFrequency = 2*np.pi*dataFrame['Frequency']

    if 'R Avg.' in dataFrame.columns and 'C Avg.' in dataFrame.columns:
        # calculate the denominator
        denomiator = 1 + (angularFrequency**2)*(dataFrame["R Avg."]**2)*(dataFrame["C Avg."]**2)

        # calulate the real part of the impedance
        z_real = dataFrame["R Avg."] / denomiator 
        z_imag = angularFrequency*(dataFrame["R Avg."]**2)*dataFrame["C Avg."]/denomiator
            
        dataFrame['Real Resistivity'] = z_real * area / thickness  # Real impedance unit Ω·mm
        dataFrame['Imag Resistivity'] = z_imag * area / thickness  # Imaginary impedance unit Ω·mm
        
        plt.figure(figsize = (6, 5))  # Set figure size
        plt.plot(dataFrame['Real Resistivity'], dataFrame['Imag Resistivity'], 'o-')  # Plot as line + points
        plt.xlabel("Z' (Ω·mm)")   # X-axis label
        plt.ylabel("Z'' (Ω·mm)")  # Y-axis label
        plt.title("Impedance Plot (Z' vs Z'')")  # Title
        plt.grid(True)  # Enable grid
        plt.tight_layout()  # Adjust spacing

        plt.show()  # Display plot
ComplexResistivity(thickness, diameter, dataFrame)

In [ ]:
# define material parameters
thickness = float(input("Enter sample thickness (mm): "))   # Thickness input
diameter = float(input("Enter sample diameter (mm): "))     # Diameter input
radius = diameter/2                                    # Radius calculation
area = np.pi*radius**2                              # Calculate sample area (circular sample)


        angularFrequency = 2 * np.pi * freq  # Angular frequency
        results_dict[freq]["angularFrequency (rad/s)"] = angularFrequency
        dataFrame["R Avg."] = results_dict[freq].get("dataFrame["R Avg."]_X")  # Retrieve dataFrame["R Avg."]
        dataFrame["C Avg."] = results_dict[freq].get("dataFrame["C Avg."]_X")  # Retrieve dataFrame["C Avg."]
        if dataFrame["R Avg."] is not None and dataFrame["C Avg."] is not None:
            try:
                
            except:
                pass  # Ignore error if calculation fails

    # Save to CSV
    with open(filename, mode='w', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=fieldnames + [
            "angularFrequency (rad/s)", "Z_real (Ω)", "Z_imag (Ω)",
            "Z_real (Ω·mm)", "Z_imag (Ω·mm)"
        ], extrasaction='ignore')
        writer.writeheader()  # Write header
        for freq in frequencies:
            writer.writerow(results_dict[freq])  # Write row by row

    print(f"\n'{filename}' has been saved!")
    print("Measurement interrupted or completed, data saved.")

    # Plot and save graph (Z'_real vs Z''_imag)
    z_real_vals = [results_dict[f].get("Z_real (Ω·mm)") for f in frequencies if results_dict[f].get("Z_real (Ω·mm)") is not None]
    z_imag_vals = [results_dict[f].get("Z_imag (Ω·mm)") for f in frequencies if results_dict[f].get("Z_imag (Ω·mm)") is not None]

    if z_real_vals and z_imag_vals:
        plt.figure(figsize=(6, 5))  # Set figure size
        plt.plot(z_real_vals, z_imag_vals, 'o-')  # Plot as line + points
        plt.xlabel("Z' (Ω·mm)")   # X-axis label
        plt.ylabel("Z'' (Ω·mm)")  # Y-axis label
        plt.title("Impedance Plot (Z' vs Z'')")  # Title
        plt.grid(True)  # Enable grid
        plt.tight_layout()  # Adjust spacing
        plt.savefig(image_filename)  # Save as image
        plt.show()  # Display plot